In [ ]:
# clone binary-classification-vMYield repo
!git clone https://github.com/lbborkowski/binary-classification-vMYield.git
%cd /content/binary-classification-vMYield/

In [ ]:
# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from numpy import loadtxt

In [ ]:
# total number of train and test data
trainArrSize=500000

In [ ]:
# create random array of length trainArrSize for three stress values (sig_rr, sig_tt, sig_rt)
sigArr=np.random.rand(trainArrSize,3)

In [ ]:
# range of stress components sig_rr, sig_tt, sig_rt obtained from running Octave file PlateWithHole.m
# this is used to create bounds for the training data
range_sig_rr=np.array([-0.03809,0.9024])
range_sig_tt=np.array([-1,3])
range_sig_rt=np.array([-0.6661,0.6661])

In [ ]:
# extend range by 1% beyond max and min to ensure all possible training stress values are sampled
range_sig_rrExt=1.01*range_sig_rr
range_sig_ttExt=1.01*range_sig_tt
range_sig_rtExt=1.01*range_sig_rt

In [ ]:
# create array of random numbers between max and min stress values using following formula:
# r = a + (b-a).*rand(N,1) where a and b are the min and max bounds, respectively
sigArr[:,0]=range_sig_rrExt[0]+(range_sig_rrExt[1]-range_sig_rrExt[0])*sigArr[:,0]
sigArr[:,1]=range_sig_ttExt[0]+(range_sig_ttExt[1]-range_sig_ttExt[0])*sigArr[:,1]
sigArr[:,2]=range_sig_rtExt[0]+(range_sig_rtExt[1]-range_sig_rtExt[0])*sigArr[:,2]

In [ ]:
# check to ensure min and max of random array extend beyond stress component ranges
if np.amin(sigArr[:,0])>range_sig_rr[0] or np.amax(sigArr[:,0])<range_sig_rr[1]:
  print('Training data does not extend beyond sig_rr range. Rerun with larger training array size')
  raise SystemExit("Execution halted")
elif np.amin(sigArr[:,1])>range_sig_tt[0] or np.amax(sigArr[:,1])<range_sig_tt[1]:
  print('Training data does not extend beyond sig_tt range. Rerun with larger training array size')
  raise SystemExit("Execution halted")
elif np.amin(sigArr[:,2])>range_sig_rt[0] or np.amax(sigArr[:,2])<range_sig_rt[1]:
  print('Training data does not extend beyond sig_rt range. Rerun with larger training array size')
  raise SystemExit("Execution halted")

In [ ]:
# assign training/test array columns to appropriate stress variables in cylindrical coordinate system
sig_rr=sigArr[:,0]
sig_tt=sigArr[:,1]
sig_rt=sigArr[:,2]

In [ ]:
# calculate von Mises stress
sig_vM=np.sqrt(sig_rr**2-sig_rr*sig_tt+sig_tt**2+3*sig_rt**2)

In [ ]:
# calculate yield based on von Mises yield criterion
# yield occurs when von Mises stress is equal to or greater than 115% of applied stress
yld=np.zeros(sig_vM.shape)
yld[sig_vM>=1.15]=1

In [ ]:
# build the binary classification model
# input: stress values (3)
# output: yield (0 or 1) - i.e., binary classification problem

In [ ]:
# separate train and test data (80/20 split)
sigArr_train=sigArr[:int(trainArrSize*0.8)]
sigArr_test=sigArr[int(trainArrSize*0.8):]
yld_train=yld[:int(trainArrSize*0.8)]
yld_test=yld[int(trainArrSize*0.8):]

In [ ]:
# define the model
model = Sequential()
model.add(Dense(12, input_dim=3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# print summary of model including total number of parameters
model.summary()

In [ ]:
# fit model using training data
%%time
# fit the model on the dataset without progress bars
model.fit(sigArr_train, yld_train, epochs=2, batch_size=5, verbose=0)

In [ ]:
# evaluate the model training accuracy
_, accuracy_train = model.evaluate(sigArr_train, yld_train, verbose=0)

In [ ]:
print('Training accuracy: %.2f%%' % (accuracy_train*100))

In [ ]:
# evaluate the model testing accuracy
_, accuracy_test = model.evaluate(sigArr_test, yld_test, verbose=0)

In [ ]:
print('Testing accuracy: %.2f%%' % (accuracy_test*100))

In [ ]:
# load 2D Cartesian coordinates of nodes in the plate
nodes_plate=loadtxt('nodes.txt', delimiter=',')

In [ ]:
# load stress values for each of the three components (sig_rr, sig_tt, sig_rt) at every node in the plate
stress_plate=loadtxt('stress.txt', delimiter=',')

In [ ]:
# load the yield value (0 or 1) for every node in the plate
yield_plate=loadtxt('yield.txt', delimiter=',')

In [ ]:
# now run the previously fit model on the plate data (validation data) to 
# evaluate the model validation accuracy
_, accuracy_valid = model.evaluate(stress_plate, yield_plate, verbose=0)

In [ ]:
print('Validation accuracy: %.2f%%' % (accuracy_valid*100))

In [ ]:
# make prediction using model for plate with a hole problem
# model will predict which nodes have yielded to compare with the analytical (baseline) solution
predictions_valid = model.predict_classes(stress_plate)

In [ ]:
# plot comparison between analytical model (baseline) and the neural network model
%matplotlib inline
plt.figure(figsize=(26,10))
plt.subplot(1, 2, 1)
plt.scatter(nodes_plate[:,0],nodes_plate[:,1],c=yield_plate)
plt.axis('off')
plt.title('Baseline model',fontsize=20)
plt.jet()
plt.colorbar()
plt.subplot(1, 2, 2)
plt.scatter(nodes_plate[:,0],nodes_plate[:,1],c=predictions_valid)
plt.axis('off')
plt.title('Neural network model',fontsize=20)
plt.jet()
plt.colorbar()
plt.show()